## Load Data Set

In [1]:
import zipfile
import os
import numpy as np
import pathlib
import pandas as pd
from math import ceil
import tensorflow as tf
import numpy as np
import IPython.display as display
import keras
from keras import backend as K
from matplotlib import pyplot as plt
from keras.utils import to_categorical


print(tf.__version__)


mnist = keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

Using TensorFlow backend.


1.13.1
11493376/11490434 [==============================] - 0s 0us/step


In [2]:
print(x_train.shape, y_train.shape)
x_train = np.expand_dims(x_train, -1)
y_train = to_categorical(y_train)
print(x_train.shape, y_train.shape)

(60000, 28, 28) (60000,)
(60000, 28, 28, 1) (60000, 10)


In [0]:
def custom_loader():
  BATCH_SIZE = 32

  trainset_size = x_train.shape[0]

  p = 0
  while True:

    idx_from = (p * BATCH_SIZE) % trainset_size

    idx_to = idx_from + BATCH_SIZE

    batch_x = x_train[idx_from: idx_to]
    batch_y = y_train[idx_from: idx_to]
    
    p += 1
  
    yield batch_x, batch_y

In [4]:
custom_gen = custom_loader()
for i in range(3):
  batch_x, batch_y = next(custom_gen)
  print(batch_x.shape, batch_y.shape)

(32, 28, 28, 1) (32, 10)
(32, 28, 28, 1) (32, 10)
(32, 28, 28, 1) (32, 10)


## Model Design

In [5]:
enc_len = 0
age_len = 10
img_shape = (28, 28, 1)
width, height, depth = (28, 28, 1)
img_len = np.prod(img_shape)
latent_dim = enc_len + age_len + img_len
noise_len = 16  # 32 x 32 x 3
input_dim = enc_len + age_len + noise_len
cond_len = enc_len + age_len


def build_discriminator():
    model = keras.Sequential([
        # dense 1
        keras.layers.Dense(144, input_shape=(latent_dim,)),
        keras.layers.Activation(tf.nn.relu),
        keras.layers.Dropout(0.2),
        
        # output
        keras.layers.Dense(1),
        keras.layers.Activation(tf.nn.sigmoid),
    ])
    
    # condition
#     c1 = keras.layers.Input(shape=(enc_len,))
    c2 = keras.layers.Input(shape=(age_len,))
    
    # image
    z = keras.layers.Input(shape=img_shape)
    
    # flatten image
    z_flat = keras.layers.Flatten()(z)
    
    # concatenation
    inputs = keras.layers.concatenate([c2, z_flat])
    
    # real or fake
    outputs = model(inputs)
    
    return keras.models.Model([c2, z], outputs)


def build_generator():
    model = keras.Sequential([
        
        # dense 1
        keras.layers.Dense(144, input_shape=(input_dim,)),
        keras.layers.Activation(tf.nn.relu),
        keras.layers.Dropout(0.2),
        
#         # reshape 1d to 3d
#         keras.layers.Reshape((12, 12, 1)),
        
#         # transpose conv block 1
#         keras.layers.Conv2DTranspose(filters=1, kernel_size=(3, 3), padding='same'),
#         keras.layers.Activation(tf.nn.relu),
#         keras.layers.UpSampling2D(size=(2,2)),
#         keras.layers.Dropout(0.2),
        
#         # transpose conv block 2
#         keras.layers.Conv2DTranspose(filters=2, kernel_size=(3, 3), padding='same'),
#         keras.layers.Activation(tf.nn.relu),
#         keras.layers.UpSampling2D(size=(2,2)),
#         keras.layers.Dropout(0.2),
        
#         # flatten
#         keras.layers.Flatten(),
        
        # dense 3
        keras.layers.Dense(784),
        keras.layers.Activation(tf.nn.relu),
        keras.layers.Dropout(0.2),
        
        # reshape 1d to 3d
        keras.layers.Reshape((width, height, depth)),
        
        # output
        keras.layers.Activation(tf.nn.tanh),
    ])
    
    # condition
#     c1 = keras.layers.Input(shape=(enc_len,))
    c2 = keras.layers.Input(shape=(age_len,))
    
    # noise
    x = keras.layers.Input(shape=(noise_len,))

    # concatenation
    inputs = keras.layers.concatenate([c2, x])
    
    # real or fake
    outputs = model(inputs)
    
    return keras.models.Model([c2, x], outputs)


generator = build_generator()
discriminator = build_discriminator()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [6]:
generator.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 10)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 16)           0                                            
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 26)           0           input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 28, 28, 1)    3617374     concatenate_1[0][0]              
Total para

In [7]:
discriminator.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 10)           0                                            
__________________________________________________________________________________________________
flatten_2 (Flatten)             (None, 784)          0           input_4[0][0]                    
__________________________________________________________________________________________________
concatenate_2 (Concatenate)     (None, 794)          0           input_3[0][0]                    
                                                                 flatten_2[0][0]                  
__________

## Generative Adversarial Network

In [0]:
GLR = 0.001  # generator
DLR = 0.01  # discriminator


discriminator.compile(
    optimizer=keras.optimizers.Adam(lr=DLR),
    loss=keras.losses.kullback_leibler_divergence,
    metrics=['accuracy']
)


# condition
# c1 = keras.layers.Input(shape=(enc_len,))
c2 = keras.layers.Input(shape=(age_len,))

# noise
x = keras.layers.Input(shape=(noise_len,))

# freeze discriminator
discriminator.trainable = False

# output
z = generator([c2, x])
out = discriminator([c2, z])

# GAN
gan = keras.models.Model(inputs=[c2, x], outputs=out)

gan.compile(
    optimizer=keras.optimizers.Adam(lr=GLR),
    loss=keras.losses.kullback_leibler_divergence,
    metrics=['accuracy'])

In [9]:
gan.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 10)           0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 16)           0                                            
__________________________________________________________________________________________________
model_1 (Model)                 (None, 28, 28, 1)    3617374     input_5[0][0]                    
                                                                 input_6[0][0]                    
__________________________________________________________________________________________________
model_2 (Model)                 (None, 1)            114625      input_5[0][0]                    
          

## Visualization Method

In [10]:
from google.colab import drive
import os


drive.mount('/content/gdrive', force_remount=True)

root_path = 'gdrive/My Drive/AGE-CGAN'
tgt_pth = os.path.join(root_path, 'visualize_mnist-v2')

if not os.path.exists(tgt_pth):
  os.mkdir(tgt_pth)

Mounted at /content/gdrive


In [0]:
def visualizeGAN(e, z_real, z_fake):

    fig, axes = plt.subplots(8, 8, figsize=(40, 36))

    r_real = 0
    r_fake = 0
    for row, axe in enumerate(axes):
        for col, cell in enumerate(axe):
            if row % 2 == 0:
                cell.imshow(z_real[r_real * 8 + col])
            else:
                cell.imshow(z_fake[r_fake * 8 + col])

            cell.axis("off")

        if row % 2 == 0:
            r_real += 1
        else:
            r_fake += 1

    plt.axis("off")
    
    fig.savefig(os.path.join(tgt_pth, '{}.jpg'.format(str(e).zfill(3))))

## Train Model

In [0]:
BATCH_SIZE = 32
EPOCHS = 100
STEPS = 60000 // BATCH_SIZE


train_loss_g = []
train_loss_d = []

train_acc_g = []
train_acc_d = []


y_fake = tf.zeros((BATCH_SIZE,))
y_true = tf.ones((BATCH_SIZE,))


config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allocator_type = 'BFC'
config.gpu_options.per_process_gpu_memory_fraction = 0.40

with tf.Session(config=config) as sess:

  K.set_session(sess)
  
  tf.initialize_all_variables().run()

  # run once
  y_fake = y_fake.eval()
  y_true = y_true.eval()

  # epochs
  for e in range(EPOCHS):

    #batches
    loss = []
    acc = []

    for p in range(STEPS * 4):

      batch_x, batch_y = next(custom_gen)

      sz = batch_x.shape[0]

      if sz != BATCH_SIZE:
        continue

      # train discriminator

      # fake data
#       c1 = row['enc']
      c2 = tf.cast(batch_y, tf.float32).eval()
      x = tf.random.normal((sz, noise_len,)).eval()
      z_fake = generator.predict([c2, x])

      # real data
#       c1 = row['enc']
      c2 = tf.cast(batch_y, tf.float32).eval()
      z_real = batch_x

      # train
      loss_1, acc_1 = discriminator.train_on_batch([c2, z_fake], y_fake)
      loss_2, acc_2 = discriminator.train_on_batch([c2, z_real], y_true)

      batch_loss = 0.5 * (loss_1 + loss_2)
      batch_acc = 0.5 * (acc_1 + acc_2)

      loss.append(batch_loss)
      acc.append(batch_acc)
    
    train_loss_d.append(np.mean(np.array(loss)))
    train_acc_d.append(np.mean(np.array(acc)))

    #batches
    loss = []
    acc = []

    for p in range(STEPS):

      batch_x, batch_y = next(custom_gen)

      sz = batch_x.shape[0]

      if sz != BATCH_SIZE:
        continue

      # train generator

      # concatenate face + age + noise
#       c1 = row['enc']
      c2 = tf.cast(batch_y, tf.float32).eval()
      x = tf.random.normal((sz, noise_len,)).eval()

      # train
      loss_1, acc_1 = gan.train_on_batch([c2, x], y_true)

      loss.append(loss_1)
      acc.append(acc_1)

    train_loss_g.append(np.mean(np.array(loss)))
    train_acc_g.append(np.mean(np.array(acc)))


    print("Epoch: {}, Steps: {}, Discriminator Accuracy: %{:.2f}, GAN Accuracy: %{:.2f}".format(
          e,
          STEPS,
          train_acc_d[-1] * 100,
          train_acc_g[-1] * 100
      ))
 
#     if e % 10 == 0:

#       pth = os.path.join(models_path, 'gan.h5')
#       gan.save(pth)

#       pth = os.path.join(models_path, 'generator-{}-{}-{}.h5'.format(e, train_loss_g[-1], train_acc_g[-1]))
#       generator.save(pth)

#       pth = os.path.join(models_path, 'discriminator.h5')
#       discriminator.save(pth)
    
    ## visualize results
    batch_x, batch_y = next(custom_gen)

    sz = batch_x.shape[0]

    # fake data
#     c1 = row['enc']
    c2 = tf.cast(batch_y, tf.float32).eval()
    x = tf.random.normal((sz, noise_len,)).eval()
    z_fake = generator.predict([c2, x])
    z_fake = np.squeeze(z_fake, axis=-1)
    
    # real data
    z_real = batch_x
    z_real = np.squeeze(z_real, axis=-1)
    
    print(z_real.shape, z_fake.shape)
    visualizeGAN(e, z_real, z_fake)

        
sess.close()

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


## Plot Loss

In [0]:
plt.figure(figsize=(20, 18))
plt.plot(train_loss_g, label="Generator Loss");
plt.plot(train_loss_d, label="Discriminator Loss");
plt.legend();

## Plot Accuracy

In [0]:
plt.figure(figsize=(20, 18))
plt.plot(train_acc_g, label="Generator Accuracy");
plt.plot(train_acc_d, label="Discriminator Accuracy");
plt.legend();

In [0]:
## Visualize Generator Outputs

In [0]:
with tf.Session() as sess:

    tf.initialize_all_variables().run()

    values = sess.run([variable_dataset])
    row = values[0]

    sz = row['img'].shape[0]

    # fake data
    c1 = row['enc']
    c2 = tf.cast(row['age'], tf.float32).eval()
    x = tf.random.normal((sz, noise_len,)).eval()
    z_fake = generator.predict([c1, c2, x])
    
    # real data
    z_real = tf.reshape(tf.io.decode_raw(row['img'], tf.uint8), (-1, width, height, depth)).eval()

sess.close()

In [0]:
fig, axes = plt.subplots(8, 8, figsize=(40, 36))

r_real = 0
r_fake = 0
for row, axe in enumerate(axes):
    for col, cell in enumerate(axe):
        if row % 2 == 0:
            cell.imshow(z_real[r_real * 8 + col])
        else:
            cell.imshow(z_fake[r_fake * 8 + col])

        cell.axis("off")
    
    if row % 2 == 0:
        r_real += 1
    else:
        r_fake += 1

plt.axis("off")
plt.show();

if not os.path.exists('./visuals'):
    os.mkdir('./visuals')
fig.savefig('./visuals/{}.jpg'.format(10))